# Generate the score report
Extract score for each sheet and convert GitHub Repo name to student ID

In [ ]:
base_folder = "data/ITE3101_practical_tests/ite-3101-practical-test-ab-submissions/"
human_score_review = "human_score_review (completed).xlsx"
assignment_name = "ite-3101-practical-test-ab"
student_id_github = "GitHub.xlsx"

In [ ]:
%pip install pandas

In [ ]:
import pandas as pd

# Load the Excel file
excel_file = pd.ExcelFile(base_folder + human_score_review)
student_df = pd.read_excel(base_folder + student_id_github)

# Student ID is string
student_df['Student ID'] = student_df['Student ID'].astype(str)

# Create an empty dictionary to store the sheet dataframes
sheet_dict = {}

# Iterate over each sheet in the Excel file
for sheet_name in excel_file.sheet_names:
    # Read the sheet data into a dataframe
    sheet_df = excel_file.parse(sheet_name)
    # Keep the Directory and Score columns
    sheet_df = sheet_df[['Directory', 'Score']]

    # Remove the assignment name from the directory
    sheet_df['Directory'] = sheet_df['Directory'].str.replace(
        assignment_name+"-", '')
    # rename Directory to GitHub Name
    sheet_df = sheet_df.rename(columns={'Directory': 'GitHub Name'})
    # # join the dataframes on GitHub Name. if GitHub Name does not exist in the student_df, it will be NaN
    # sheet_df = sheet_df.join(student_df.set_index('GitHub Name'), on='GitHub Name')

    # Rename Score to sheet_name
    sheet_df = sheet_df.rename(columns={'Score': sheet_name})

    # Add the dataframe to the dictionary with the sheet name as the key
    sheet_dict[sheet_name] = sheet_df

# Join all the dataframes in the dictionary value together into one dataframe on the GitHub Name column
df = pd.concat(sheet_dict.values(), axis=1, join='outer')
# Keep the first GitHub Name column
df = df.iloc[:, ~df.columns.duplicated()]

In [ ]:
df['Score'] = df.drop('GitHub Name', axis=1).sum(axis=1)

# join the dataframes on GitHub Name. if GitHub Name does not exist in the student_df, it will be NaN
df = df.join(student_df.set_index('GitHub Name'), on='GitHub Name')

# Student ID to the first column, then Score to the second column
cols = df.columns.tolist()
cols = cols[-2:] + cols[:-2]
df = df[cols]
# swap the first two columns
cols = df.columns.tolist()
cols = cols[1:2] + cols[0:1] + cols[2:]
df = df[cols]
df.head()

In [ ]:
df.to_excel(base_folder+assignment_name+".xlsx", index=False)